In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
from IPython.display import Markdown, display, update_display

In [4]:
load_dotenv(override=True)
openai_api_key=os.getenv("OPENAI_API_KEY")
gemini_api_key=os.getenv("GOOGLE_API_KEY")

if openai_api_key:
    print("OpenAI api key successfully loaded...")
else:
    print("OpenAI api key missing...")

if gemini_api_key:
    print("Gemini api key successfully loaded...")
else:
    print("Gemini api key missing...")

OpenAI api key successfully loaded...
Gemini api key successfully loaded...


In [20]:
system_prompt="""
You are an IT Security Analyst specializing in incident response and log analysis. 
Your role is to review incoming log data, identify potential security threats, and provide a clear, concise incident report.

When analyzing logs:
- Focus only on entries that indicate a real or likely threat (ignore normal/noise logs).
- For each flagged incident, explain *why it is suspicious* and *what action should be taken*.
- Use precise, professional language that could be dropped directly into a SOC ticket.

⚠️ Response Requirement:
- You must always format the response in **Markdown**.
- Use headings, bullet points, and bold text for clarity.
- Use fenced code blocks (```) for raw log data.
- Keep reports readable and structured for analysts.

Your response must always follow this format:

### Incident Report
**Log Data:**  [Insert relevant log lines]
**Timeframe:** [Insert timeframe of event]  
**Threat Indicators:** [List IOCs, errors, anomalies, suspicious patterns]  
**Analysis:** [Brief explanation of why this is a threat / what it suggests]  
**Recommended Actions:** [Clear, prioritized next steps for containment, investigation, or remediation]  
**Severity Level:** [Low | Medium | High | Critical]

Keep the report concise (bullet points where possible). 
Do not add extra commentary outside the structured report.
"""


In [38]:
openAI=OpenAI()
gpt_model="gpt-4o-mini"

def call_model(incident:str):
    response=openAI.chat.completions.create(
        model=gpt_model,
        messages=[
            {"role":"system", "content": system_prompt},
            {"role":"user", "content":incident}
        ],
    )
    
    display(Markdown(response.choices[0].message.content))

In [39]:

incident_1="""
2025-08-21 14:03:17 [web-app] [WARN] Query failed: 
SELECT * FROM users WHERE username = 'admin'--' AND password = 'password';
"""

incident_2="""
2020-10-25 21:45:13 [orion-srv01] [netflow] 
Outbound connection established
Source: 10.0.5.15:8080 
Destination: 104.244.42.129:443 (unknown-host.net)
Protocol: TLS1.2
"""

In [40]:
call_model(incident_1)

### Incident Report
**Log Data:**  
```
2025-08-21 14:03:17 [web-app] [WARN] Query failed: 
SELECT * FROM users WHERE username = 'admin'--' AND password = 'password';
```
**Timeframe:** 2025-08-21 14:03:17  
**Threat Indicators:**  
- SQL injection attempt indicated by improper comment usage (`--`)  
- Use of a common username (`admin`) and weak password in query  
- Suspicious query structure aimed at revealing user credentials  

**Analysis:**  
This log entry shows a failed SQL query that is likely part of an attempted SQL injection attack. The presence of `--`, which in SQL comments out the rest of the line, indicates an attacker attempting to manipulate the SQL statement to bypass authentication. Given the commonality of the username and the attempt to keep the password field as a comment, this raises significant concerns regarding the integrity of the application's input handling and potential exposure of sensitive user data.

**Recommended Actions:**  
- Immediately validate input sanitization and prepared statement usage in the web application to protect against SQL injection attacks.  
- Monitor and analyze incoming web traffic patterns to identify additional malicious attempts.  
- Increase logging levels to capture more details on failed login attempts and SQL queries.  
- Conduct a security assessment of the web application's database interaction mechanisms.  
- Consider implementing web application firewall (WAF) rules to mitigate exposure to SQL injection.

**Severity Level:** High

In [41]:
call_model(incident_2)

### Incident Report
**Log Data:**  
```
2020-10-25 21:45:13 [orion-srv01] [netflow] 
Outbound connection established
Source: 10.0.5.15:8080 
Destination: 104.244.42.129:443 (unknown-host.net)
Protocol: TLS1.2
```
**Timeframe:** 2020-10-25 21:45:13  
**Threat Indicators:**  
- Outbound connection to an unknown host (104.244.42.129)  
- Use of a non-standard port (8080) for outbound communications  
- Connection established over TLS1.2, which may suggest encrypted traffic with malicious intent  

**Analysis:**  
- The connection to 104.244.42.129 raises suspicion as this is an unknown entity without any established reputation.
- The use of port 8080 may indicate that this connection bypasses typical security measures, often associated with web proxies or malicious applications.
- Outbound connections to unrecognized IP addresses can signify data exfiltration or communications between compromised hosts.

**Recommended Actions:**  
- Block outbound traffic to 104.244.42.129 immediately. 
- Perform a detailed investigation on the host at 10.0.5.15 to identify any malware or unauthorized applications.
- Review security policies to assess potential weaknesses and ensure compliance.
- Monitor logs for further suspicious outbound connections or anomalous behaviors.

**Severity Level:** High